# Lab 2: Función de Scoring 

En esta práctica vamos a profundizar en el funcionamiento de scoring de las consultas de Eleasticsearch y como lo utiliza para asignar la relevancia de los documentos resultantes de una búsqueda.

Puesto que es un concepto clave para entender correctamente la ordenación de los resultados de las consultas de Elasticsearch y una funcionalidad muy interesante para la implementación de buscadores sobre Elasticsearch, vamos a dedicarle el tiempo necesario para profundizar y entenderlo correctamente.



## 1. Función de puntuación

Puesto que la base del scoring en Elasticserach es la función de puntuación que emplea para puntuar los diferentes resultados o match de la consulta, vamos a verla en detalle.

Hasta la versión 2.0 de Eleasticsearch se tulizaba la clásica función de similaridad TF*IDF a partir de dicha versión utiliza una nueva función de similaridad llamada BM25 que también utiliza los conceptos de TF e IDF.

La función de puntuación BM25 se define de la siguiente manera:

<img src="../../images/els/bm25_equation.png" alt="index management"/>

Donde:
1. **qi** es el i-esimo término de la consutla.
2. **IDF(qi)** es la inversa de la frecuencia de documento del término i-esimo de la consulta.
    * La función **IDF** evalúa la frecuencia con la que un término aparece en todos los documentos y penalíza los términos que son comunes.
    * La fórmuna actual de BM25 es la siguiente:
    <img src="../../images/els/idf_equation.png" alt="index management"/>
    
        * **docCount** es el número de documentos que tienen un valor para el cámpo donde se está buscando el término.
        * **f(qi)** es el número de documentos que contienene el término i-ésimo.
3. **fieldLen/avgFieldLen** Intenta reflejar la relevancia de un término en para un campo:
    * **fieldLen** es la logitud de un campo, es decir el número de términos que contiene.
    * **avgFieldLen** es la longitud media de un campo teniendo en cuenta todos los documentos.
    * Un campo que contenga menos términos se considera más relvante que un campo que tenga muchos términos. si un documento es más largo que la media, entoces el denominador crece, decreciendo la puntuación y si es más pequeño que la media, entonces el denominador se hace más pequeño, lo que hace incrementar la puntuación. 
    
4. **Parámetro b en el denominador** Permite modificar la influencia de factor longitud del campo en el resultado. Por ejemplo, cuanto más aumente su valor, más se aplifica la influencia del factor de longitud. Si por el contrario se iguala a 0 el factor de longitud deja de tener relevancia en al puntuación.
    * Su valor por defecto es de 0.75
    
5. **f(qi,D)** es el número de veces que aparece el término i-ésimo de la consulta en el documento D. Cuanta más veces apareza el término qi en el documento más relevante se considera y por tanto más puntuación tendrá.

6. **k1** variable que nos permite indicar el grado de saturación para la fercuencia de un término:
    * Nos ayuda a determinar como un único término puede afectar a la puntuación de un documento.
    * El valor por defecto es de 1.2

Podemos reformular la función anterior en términos de IDF y TF para un término de la siguiente forma:

`
score(qi) = IDF(qi) * TF(qi) 
`

Donde:
* **TF** es la frecuencia de un término i-ésimo del consulta q. Intenta reflejar la relevancia de un término dentro de un documento, es decir cuantas más apariciones de un término en un documento, más relevante se considera el documento.
* **IDF** calcula la inversa de la fecuencia de documento del término i-esimo de la consulta q. Intenta reflejar la relevancia del término sobre el conjunto de todos los documentos. Un término se considera menos relevante si aparece en muchos documentos del índcie. 

### Boosting

Podemos influir en el resultado de la función de puntuación potenciando su valor por medio de la técnica boosting. Consiste en dar un factor de multiplicación al resultado de la función de puntuación en función de si el término a buscar se encontrón en un índcie en concreto o en un campo en concreto. Por lo que la función de puntuación la prodríamos reformular de la siguiente forma:

`
score(qi) = IDF(qi) * TF(qi) * Boost
`

1. Boosting de índcie, tanto a la hora de crear un índice como a la hora de consultarlo, podemos hacer que los documentos provenientes de un indice tengan más relevancia (puntuación) en relación a otros índices potenciando dicho índice. Para ello indicaremos un valor de boost para dicho índicie. Es útil cuando realizamos consultas sobre varios índcies y queremos indicar que es más relevanta para el caso de uso los resultados provinientes de un o varios índcies en concreto.
2. Boosting de campo, tanto a la hora de crear un índice como a la hora de consultarlo, podemos modificar el resultao de la puntución indicando que la puntución sea mayor para las ocurrencias de un término en un determinado campo de un documento.


## 2. Función de puntuación en la practica

Vamos a poner todos los conceptos vistos en el apartado anterior y para ver como se comporta la función de scoring vamos a utlizar el método explain de la API de elasticserch, que nos indicará el valor para cada uno de los valores de la función de scoring.

### 2.1. Consultar la información del scoring para una consulta

Para obtener la información de como se ha generado el scoring para una consulta vamos a utilizar la petición "_explain" de la API de elasticsearch. Para utilizar esta petición y puesto que el scoring se calcula para cada documento, necesitamos indicarle el "_id" del documento para el que vamos a evaluar su puntuación. Por lo que antes de nada vamos a buscar la receta con el título "Hershey's "Perfectly Chocolate" Chocolate Cake" para buscar el "_id" de documento.

`
POST recipes/_search
{
  "query": {
    "match": {
      "title": """Hershey's "Perfectly Chocolate" Chocolate Cake"""
    }
  }
}
`

Toma el "_id" del primer documento que aparezca. En mi caso es el siguiente documento:

`
{
        "_index": "recipes",
        "_id": "quawoIQBnF462_TnMdAZ",
        "_score": 4.618042,
        "_ignored": [
          "event.original.keyword"
        ],
        "_source": {
          "picture_link": "https://assets.epicurious.com/photos/58485cfe3b047eac0f3b347e/6:4/w_274%2Ch_169/Hershey%E2%80%99s-Perfectly-Chocolate-Chocolate-Cake-07122016-1.jpg",
          "log": {
            "file": {
              "path": "/tmp/data/recipes.json"
            }
          },
          "author": "The Hershey Company",
          "url": "http://www.epicurious.com/recipes/food/views/hersheys-perfectly-chocolate-chocolate-cake"
}
`

Con el "_id" que has seleccionado, realiza la petición "_explain":

`
GET recipes/_explain/quawoIQBnF462_TnMdAZ
{
  "query": {
    "match": {
      "title": "chocolate"
    }
  }
}
`

En este caso la query que queremos evaluar es aquella que busca todos los documentos que contengan el término "chocolate" en el campo "title".

En mi caso me devuelve la signiente información:

`
{
  "_index": "recipes",
  "_id": "quawoIQBnF462_TnMdAZ",
  "matched": true,
  "explanation": {
    "value": 4.618042,
    "description": "weight(title:chocolate in 680) [PerFieldSimilarity], result of:",
    "details": [
      {
        "value": 4.618042,
        "description": "score(freq=2.0), computed as boost * idf * tf from:",
        "details": [
          {
            "value": 2.2,
            "description": "boost",
            "details": []
          },
          {
            "value": 3.3940127,
            "description": "idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:",
            "details": [
              {
                "value": 74,
                "description": "n, number of documents containing term",
                "details": []
              },
              {
                "value": 2218,
                "description": "N, total number of documents with field",
                "details": []
              }
            ]
          },
          {
            "value": 0.61847436,
            "description": "tf, computed as freq / (freq + k1 * (1 - b + b * dl / avgdl)) from:",
            "details": [
              {
                "value": 2,
                "description": "freq, occurrences of term within document",
                "details": []
              },
              {
                "value": 1.2,
                "description": "k1, term saturation parameter",
                "details": []
              },
              {
                "value": 0.75,
                "description": "b, length normalization parameter",
                "details": []
              },
              {
                "value": 5,
                "description": "dl, length of field",
                "details": []
              },
              {
                "value": 4.8192067,
                "description": "avgdl, average length of field",
                "details": []
              }
            ]
          }
        ]
      }
    ]
  }
}
`

Vamos a verlo en detalle:

Lo primero me indica el índice sobre el que se ha hecho la búsqueda y el _id del documento evaluado. También me indica que ha habido un match de ese documento para la consulta. Si no hay match no hay scoring:

`
  "_index": "recipes",
  "_id": "quawoIQBnF462_TnMdAZ",
  "matched": true,
`

El siguiente valor relevante es la puntuación obtenida para ese documento: 

`"value": 4.618042` 

y la descripción del scoring: 

`weight(title:chocolate in 680) [PerFieldSimilarity]`

Al final entraremos en detalle del significado de esta descripción, primero vamos a ver como nos dice elasticsearch que se a calculado este valor.

Primero nos indica la fórmula empleada para calcular el valor, la cual ya conocemos:

`score(freq=1.0), computed as boost * idf * tf`

Puesto que no hemos definido ningun boost para el índice ni para el campo "title" por el que estamos buscando, elasticsearch utiliza su valor por defecto 2.2:

`
{
    "value": 2.2,
    "description": "boost",
    "details": []
},
`

A continuación nos explica como ha calculado el valor para idf y el valor obtendio

`
{
    "value": 3.3940127,
    "description": "idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:",
    "details": [
      {
        "value": 74,
        "description": "n, number of documents containing term",
        "details": []
      },
      {
        "value": 2221,
        "description": "N, total number of documents with field",
        "details": []
      }
    ]
},
`
Para calcularlo nos dice que ha utilizado la siguiente función log(1 + (N - n + 0.5) / (n + 0.5)) donde n es el número de documentos que contiene el términos (74) y N es el total de documentos que tienen el campo "title" (2221)

Por último nos indica el valor para tf y cómo lo ha calculado:

`
{
    "value": 0.61847436,
    "description": "tf, computed as freq / (freq + k1 * (1 - b + b * dl / avgdl)) from:",
    "details": [
      {
        "value": 2,
        "description": "freq, occurrences of term within document",
        "details": []
      },
      {
        "value": 1.2,
        "description": "k1, term saturation parameter",
        "details": []
      },
      {
        "value": 0.75,
        "description": "b, length normalization parameter",
        "details": []
      },
      {
        "value": 5,
        "description": "dl, length of field",
        "details": []
      },
      {
        "value": 4.8192067,
        "description": "avgdl, average length of field",
        "details": []
      }
    ]
}
`

Para calcularlo ha utilizado la función que ya conocemos: freq / (freq + k1 * (1 - b + b * dl / avgdl)) donde: 
* freq es el número de ocurencias del término en el documento, en nuestro caso aparece 2 veces.
* k1 es el parametro de saturación del término que como no lo hemos modificado toma el valor por defecto 1.2
* b es el valor de normalización de la longitud del campo que como no lo hemos modificado toma el valor por defecto 0.75.
* dl es la longitud del campo en número de términos. En nuestro caso el valor del campo "title" es "Hershey's "Perfectly Chocolate" Chocolate Cake" por lo que su longitud es de 5 terminos.
* avgdl es la media de longitud en términos del campo para todos los documentos indexados. En este caso 4.8192067. 

Hasta aquí todo correcto, pero vamos a entrar aun mas en detalle para entender los valores que nos ha devuelto.

### 2.2. Sharding Effect

Volvamos a los valores n y N con los que se calcularon el valor de idf. En nuestro caso el valor de N es 1640. Si consultamos en Kibana el número de documentos indexados veremos que el total asciende a 6500. ¿Qué ha pasado con el resto de documentos?

En un principio podríamos pensar que esos documentos no tienen el campo "Title", pero como la importación la hemos hecho a partir de un csv, sabemos que ese campo existe en todos los documentos, por lo que el problema tiene que estar en otro sitio.

Cuando le importamos los datos le indicamos a Kibana que crease un índcie con 4 shards. Vamos a ver que pasa si dividimos 6500 entre 4, el resultado es de 1.625 curiosamente próximo a 1640.

Realicemos una consulta rápida:

`
GET /_cat/shards/recipes?v=true
`

En mi caso el resultado es el siguiente:

<img src="../../images/els/index_shards_recipies.png" alt="index management"/>

Como podemos ver el shard 0 tiene justo el número de documentos que indica el valor N. De aquí podemos deducir dos cosas:

* El documento está indexado en el shard 0.
* Lo que es más importante para el calculo del scooring, sólo se está teniendo en cuenta los datos del shard donde se encuentra el documento para el que genera la puntuación.

Esto es lo que se conoce con el "Sharding Effect". Cuando elasticsearch realiza una búsqieda hace un fanout de la query por todos los shards del índice y esta se ejecuta y evalúa en cada shard, ya que un shard es un índice de Lucene perfectamente funcional, devolviendo los 10 primeros _id de los documentos que machean la consulta **ordenados por su scoring**. Esto es lo que se llama la "query phase" de una consulta. Posteriormente cada shard devuelve la lista del resultado de evaluar la query en su índice al nodo orquestador que reordena estas listas de forma global y se devuelve el resultado final, lo que se conoce como la "fetch phase".

¿Cómo podemos solucionar este problema?

Cambiando el orden en el que se realiza la evaluación de los documentos utilizando el campo search_type en la búsqueda:
* El valor por defecto "search_type=query_then_fetch" evalua localmente las frecuencias de los terminos.
* El valor "search_type=dfs_query_then_fetch" permite calcular globalmente las frecuencias, pero aumenta la latencia de respuesta.
    
Vamos a ver como funciona para ejecutemos la consuta con la petición _sarch a la que le indicamos que utilice el search_type=dfs_query_then_fetch y además le pedimos que haga el explain:

`
POST recipes/_search?explain=true&pretty=true&search_type=dfs_query_then_fetch
{
  "query": {
    "match": {
      "title": "chocolate"
    }
  }
}
`

Para mi caso, en el resultado ya podemos ver como ha cambiado el resultado del score, en este caso el valor N ya si es igual a 6500:

`
"_explanation": {
    "value": 4.414281,
    "description": "weight(title:chocolate in 590) [PerFieldSimilarity], result of:",
    "details": [
    {
      "value": 4.414281,
      "description": "score(freq=1.0), computed as boost * idf * tf from:",
      "details": [
        {
          "value": 2.2,
          "description": "boost",
          "details": []
        },
        {
          "value": 3.364237,
          "description": "idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:",
          "details": [
            {
              "value": 310,
              "description": "n, number of documents containing term",
              "details": []
            },
            {
              "value": 8976,
              "description": "N, total number of documents with field",
              "details": []
            }
          ]
        },
        {
          "value": 0.5964179,
          "description": "tf, computed as freq / (freq + k1 * (1 - b + b * dl / avgdl)) from:",
          "details": [
            {
              "value": 1,
              "description": "freq, occurrences of term within document",
              "details": []
            },
            {
              "value": 1.2,
              "description": "k1, term saturation parameter",
              "details": []
            },
            {
              "value": 0.75,
              "description": "b, length normalization parameter",
              "details": []
            },
            {
              "value": 2,
              "description": "dl, length of field",
              "details": []
            },
            {
              "value": 4.7786317,
              "description": "avgdl, average length of field",
              "details": []
            }
          ]
        }
      ]
    }
`



## 3. La influencia de los analizadores en la puntuación

Como hemos visto, para calcular la puntuación se está evaluando constantemente las diferentes frecuencias para los términos, por lo que la forma en la que se extraen esos términos es fundamental para conseguir una buena ordenación por relevancia.

* Utilizar el analizador del idioma en el que están escritos los textos de un documento ayudará a generar mejores términos y por tanto calcular con mayor precisión la puntución de los resultados de las consutlas.
* Utilizar steamers que ayudan a mejorar el cálculo de las frecuencia de los términos al tener en cuenta los lexemas y raíces semánticas de los términos. 
* Usar listas de sinónimos ayudará a identificar terminos "equivalentes" lo que permitirá calcular con mayor precisión las frecuencias.
* Definir correctos filtros para eliminar términos no relevantes o no tener en cuenta mayúsculas y minúsclas ayudan a eliminar ruido en el cálculo de la relevancia.
* No solo definir analizadores a la hora de indexar los documentos, sino también a la hora de analizar los términos de la consulta que queremos realizar.